# Import packages, modules, ...

In [ ]:
import pandas as pd
import numpy as np
import random
import os
import cv2
import h5py
from datetime import datetime
from tqdm import tqdm

import tensorflow as tf
import keras
import keras.backend as K
import keras.callbacks
from keras.models import Model
from keras.models import load_model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Dropout, concatenate

# Define Utilities

In [ ]:
def mask2rle(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels= img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)
 
def rle2mask(mask_rle, shape=(1600,256)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (width,height) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T

# Define generator

In [ ]:
class CSVGenerator(keras.utils.Sequence):
    def __init__(
        self,
        dataset_dir,
        batch_size=1,
        group_method='random',  # one of 'none', 'random'
        shuffle_groups=True
    ):
        """ Initialize Generator object.

        Args
            batch_size             : The size of the batches to generate.
            group_method           : Determines how images are grouped together (defaults to 'ratio', one of ('none', 'random')).
            shuffle_groups         : If True, shuffles the groups each epoch.
        """
        self.batch_size = int(batch_size)
        self.group_method = group_method
        self.shuffle_groups = shuffle_groups

        self.dataset_dir = dataset_dir
        self.df = pd.read_csv(os.path.join(self.dataset_dir, 'train.csv'))
        self.image_files = os.listdir(os.path.join(self.dataset_dir, 'train_images'))
        self.image_ids = range(len(self.image_files))

        # self.load_classes()

        # Define groups
        self.group_images()

        # Shuffle when initializing
        if self.shuffle_groups:
            self.on_epoch_end()

    def on_epoch_end(self):
        if self.shuffle_groups:
            random.shuffle(self.groups)

    def size(self):
        """ Size of the COCO dataset.
        """
        return len(self.image_ids)

    def load_image(self, image_index):
        """ Load an image at the image_index.
        """

        path = os.path.join(self.dataset_dir, 'train_images', self.image_files[image_index])

#         with rasterio.open(path) as src:
#             image = src.read().transpose(1, 2, 0)

#         image = cv2.imread(path, 1)
#         image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        image = cv2.imread(path, 0)
        image = image.reshape(256, 1600, 1)

        return image

    def load_mask(self, image_index):
        """ Load mask for an image_index.
        """

        # Search for annotations
        labels = self.df[self.df['ImageId_ClassId'].str.contains(self.image_files[image_index])]['EncodedPixels']

        # 5 classes: 0-background, 1-1st class, 2-2nd class, 3-3th class, 4-4th class
        masks = np.zeros((256, 1600, 5), dtype=np.uint8)

#         for idx, label in enumerate(labels.values):
#             if label is not np.nan:
#                 label = label.split(" ")
#                 positions = map(int, label[0::2])
#                 length = map(int, label[1::2])
#                 mask = np.zeros(256 * 1600, dtype=np.uint8)
#                 for pos, le in zip(positions, length):
#                     mask[pos:(pos + le)] = 1
#                 masks[:, :, idx] = mask.reshape(256, 1600, order='F')
        background = np.zeros((256, 1600), dtype=np.uint8)
        for idx, rle in enumerate(labels.values):
            if rle is not np.nan:
                masks[:, :, idx+1] = rle2mask(rle)
                background += masks[:, :, idx+1]
                
        np.clip(background, 0, 1, background)
        background = np.ones((256, 1600), dtype=np.uint8) - background
        masks[:, :, 0] = background

        return masks

    def load_image_group(self, group):
        """ Load images for all images in a group.
        """
        return np.array([self.load_image(image_index) for image_index in group])

    def load_mask_group(self, group):
        """ Load annotations for all images in group.
        """
        mask_group = np.array([self.load_mask(image_index) for image_index in group])

        return mask_group

    def preprocess_group_entry(self, image):
        """ Preprocess image.
        """
        # preprocess the image
        image = image / 255.

        # convert to the wanted keras floatx
        # image = keras.backend.cast_to_floatx(image)

        return image

    def preprocess_group(self, image_group):
        """ Preprocess each image in its group.
        """
        for index in range(len(image_group)):
            # preprocess a single group entry
            image_group[index] = self.preprocess_group_entry(image_group[index])

        return image_group

    def group_images(self):
        """ Order the images according to self.order and makes groups of self.batch_size.
        """
        # determine the order of the images
        order = list(range(self.size()))
        if self.group_method == 'random':
            random.shuffle(order)

        # divide into groups, one group = one batch
        self.groups = [[order[x % len(order)] for x in range(i, i + self.batch_size)] for i in range(0, len(order), self.batch_size)]

    def compute_input_output(self, group):
        """ Compute inputs and target outputs for the network.
        """
        # load images and annotations
        image_group = self.load_image_group(group)
        mask_group = self.load_mask_group(group)

        assert(len(image_group) == len(mask_group))

        # randomly transform data
        # image_group, annotations_group = self.random_transform_group(image_group, annotations_group)

        # perform preprocessing steps
        image_group = self.preprocess_group(image_group)


        return image_group, mask_group

    def __len__(self):
        """
        Number of batches for generator.
        """

        return len(self.groups)

    def __getitem__(self, index):
        """
        Keras sequence method for generating batches.
        """
        group = self.groups[index]
        inputs, targets = self.compute_input_output(group)

        return inputs, targets

# Define losses

In [ ]:
def binary_focal_loss(gamma=2., alpha=.25):
    """
    Binary form of focal loss.

      FL(p_t) = -alpha * (1 - p_t)**gamma * log(p_t)

      where p = sigmoid(x), p_t = p or 1 - p depending on if the label is 1 or 0, respectively.

    References:
        https://arxiv.org/pdf/1708.02002.pdf
    Usage:
     model.compile(loss=[binary_focal_loss(alpha=.25, gamma=2)], metrics=["accuracy"], optimizer=adam)

    """
    def binary_focal_loss_fixed(y_true, y_pred):
        """
        :param y_true: A tensor of the same shape as `y_pred`
        :param y_pred:  A tensor resulting from a sigmoid
        :return: Output tensor.
        """
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))

        epsilon = K.epsilon()
        # clip to prevent NaN's and Inf's
        pt_1 = K.clip(pt_1, epsilon, 1. - epsilon)
        pt_0 = K.clip(pt_0, epsilon, 1. - epsilon)

        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1)) \
               -K.sum((1 - alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0))

    return binary_focal_loss_fixed


def categorical_focal_loss(gamma=2., alpha=.25):
    """
    Softmax version of focal loss.

           m
      FL = ∑  -alpha * (1 - p_o,c)^gamma * y_o,c * log(p_o,c)
          c=1

      where m = number of classes, c = class and o = observation

    Parameters:
      alpha -- the same as weighing factor in balanced cross entropy
      gamma -- focusing parameter for modulating factor (1-p)

    Default value:
      gamma -- 2.0 as mentioned in the paper
      alpha -- 0.25 as mentioned in the paper

    References:
        Official paper: https://arxiv.org/pdf/1708.02002.pdf
        https://www.tensorflow.org/api_docs/python/tf/keras/backend/categorical_crossentropy

    Usage:
     model.compile(loss=[categorical_focal_loss(alpha=.25, gamma=2)], metrics=["accuracy"], optimizer=adam)
    """
    def categorical_focal_loss_fixed(y_true, y_pred):
        """
        :param y_true: A tensor of the same shape as `y_pred`
        :param y_pred: A tensor resulting from a softmax
        :return: Output tensor.
        """

        # Scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)

        # Clip the prediction value to prevent NaN's and Inf's
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1. - epsilon)

        # Calculate Cross Entropy
        cross_entropy = -y_true * K.log(y_pred)

        # Calculate Focal Loss
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy

        # Sum the losses in mini_batch
        return K.sum(loss, axis=1)

    return categorical_focal_loss_fixed

# Define metrics

In [ ]:
def jaccard_coef(y_true, y_pred):
    smooth = 1e-12
#     intersection = K.sum(y_true * y_pred, axis=[0, -1, -2])
#     sum_ = K.sum(y_true + y_pred, axis=[0, -1, -2])
    intersection = K.sum(y_true * y_pred, axis=[1, 2, 3])
    sum_ = K.sum(y_true + y_pred, axis=[1, 2, 3])
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return K.mean(jac)


def jaccard_coef_int(y_true, y_pred):
    smooth = 1e-12
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
#     intersection = K.sum(y_true * y_pred_pos, axis=[0, -1, -2])
#     sum_ = K.sum(y_true + y_pred_pos, axis=[0, -1, -2])
    intersection = K.sum(y_true * y_pred_pos, axis=[1, 2, 3])
    sum_ = K.sum(y_true + y_pred_pos, axis=[1, 2, 3])
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return K.mean(jac)


def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


def dice_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(y_true * y_pred, axis=[1,2,3])
    union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
    return K.mean( (2. * intersection + smooth) / (union + smooth), axis=0)

# Define U-Net model

In [ ]:
def unet(input_size=(256, 256, 6)):
    kwargs = {
        'activation': 'relu',
        'padding': 'same',
        'kernel_initializer': 'he_normal'
    }

    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, **kwargs)(inputs)
    conv1 = Conv2D(64, 3, **kwargs)(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, **kwargs)(pool1)
    conv2 = Conv2D(128, 3, **kwargs)(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, **kwargs)(pool2)
    conv3 = Conv2D(256, 3, **kwargs)(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, **kwargs)(pool3)
    conv4 = Conv2D(512, 3, **kwargs)(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, **kwargs)(pool4)
    conv5 = Conv2D(1024, 3, **kwargs)(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, **kwargs)(UpSampling2D(size=(2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis=3)
    conv6 = Conv2D(512, 3, **kwargs)(merge6)
    conv6 = Conv2D(512, 3, **kwargs)(conv6)

    up7 = Conv2D(256, 2, **kwargs)(UpSampling2D(size=(2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis=3)
    conv7 = Conv2D(256, 3, **kwargs)(merge7)
    conv7 = Conv2D(256, 3, **kwargs)(conv7)

    up8 = Conv2D(128, 2, **kwargs)(UpSampling2D(size=(2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis=3)
    conv8 = Conv2D(128, 3, **kwargs)(merge8)
    conv8 = Conv2D(128, 3, **kwargs)(conv8)

    up9 = Conv2D(64, 2, **kwargs)(UpSampling2D(size=(2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis=3)
    conv9 = Conv2D(64, 3, **kwargs)(merge9)
    conv9 = Conv2D(64, 3, **kwargs)(conv9)
    # conv9 = Conv2D(2, 3, **kwargs)(conv9)
    conv10 = Conv2D(5, 1, activation='softmax')(conv9)

    model = Model(inputs=inputs, outputs=conv10)

    return model

# Define callbacks

In [ ]:
class LRTensorBoard(keras.callbacks.TensorBoard): # The callback that logs learning rate in TensorBoard
    def __init__(self,
        log_dir,
        histogram_freq,
        batch_size,
        write_graph,
        write_grads,
        write_images,
        embeddings_freq,
        embeddings_layer_names,
        embeddings_metadata
    ):
        super().__init__(
            log_dir=log_dir,
            histogram_freq=histogram_freq,
            batch_size=batch_size,
            write_graph=write_graph,
            write_grads=write_grads,
            write_images=write_images,
            embeddings_freq=embeddings_freq,
            embeddings_layer_names=embeddings_layer_names,
            embeddings_metadata=embeddings_metadata
        )

    def on_epoch_end(self, epoch, logs=None):
        logs.update({'lr': K.eval(self.model.optimizer.lr)})
        super().on_epoch_end(epoch, logs)

# Training configuration

In [ ]:
snap_path = '../input/steel-defect-detection-multi-class-u-net/model.h5'
dataset_dir = '../input/severstal-steel-defect-detection/'
batch_size = 4
steps_per_epoch = np.ceil(12568 / batch_size).astype(np.uint64)
# validation_steps = args.validation_steps
epochs = 5
lr = 1e-4
gamma = 2.0
alpha = [0.1, 0.9, 0.9, 0.9, 0.9]
input_shape = (256, 1600, 1)
num_trainable_layers = 0 # All layers are trainable
optimizer = 'Adam'
threads = 2
log_dir = './'
snap_dir = './'
histogram_freq = 0
write_graph = False
write_grads = False

# Start training

In [ ]:
def create_callbacks(batch_size, log_dir, snap_dir, input_shape, histogram_freq, write_graph, write_grads):
    tensorboard_callback = LRTensorBoard(
        log_dir=log_dir,
        histogram_freq=histogram_freq,
        batch_size=batch_size,
        write_graph=write_graph,
        write_grads=write_grads,
        write_images=False,
        embeddings_freq=0,
        embeddings_layer_names=None,
        embeddings_metadata=None
    )

    reduce_lr_on_plateau_callback = keras.callbacks.ReduceLROnPlateau(
        monitor='loss',
        factor=0.1,
        patience=2,
        verbose=1,
        mode='auto',
        min_delta=0.0001,
        cooldown=0,
        min_lr=0
    )

    checkpointer = keras.callbacks.ModelCheckpoint(
        os.path.join(snap_dir, 'model.h5'),
        monitor='loss',
        save_best_only=True,
        verbose=1
    )

    return [tensorboard_callback, reduce_lr_on_plateau_callback, checkpointer]




# Create snapshot directory if not exist
if not os.path.exists(snap_dir):
    os.makedirs(snap_dir)


# Get generators
train_generator = CSVGenerator(
    dataset_dir,
    batch_size=batch_size
)


if not snap_path:
    # Train U-Net model from scratch
    model = unet(input_shape)
else:
    print('\nLoading snapshot...')
    model = load_model(
        snap_path,
        custom_objects={
            'dice_coef': dice_coef,
            'f1': f1,
            'jaccard_coef': jaccard_coef,
            'jaccard_coef_int': jaccard_coef_int,
            'categorical_focal_loss_fixed': categorical_focal_loss(gamma=gamma, alpha=alpha)
        }
    )


# Freeze all layers except TOP n layers, where n = num_trainable_layers
for layer in model.layers:
    layer.trainable = True
for layer in model.layers[:-num_trainable_layers]:
    layer.trainable = False

print('\nThere are total {} layers'.format(len(model.layers)))
for i, layer in enumerate(model.layers[::-1]):
    print('#{}: {} - '.format(i+1, layer.__class__.__name__), end='')
    if layer.trainable:
        print('trainable')
    else:
        print('untrainable')


# Select optimizer
if optimizer == 'adam':
    optimizer = keras.optimizers.Adam
elif optimizer == 'sgd':
    optimizer = keras.optimizers.SGD


# Compile the model
model.compile(
    optimizer=keras.optimizers.Adam(lr=lr),
    loss=categorical_focal_loss(gamma=gamma, alpha=alpha),
    metrics=[dice_coef, f1, jaccard_coef, jaccard_coef_int],
)


# Create callbacks
callbacks = create_callbacks(
    batch_size,
    log_dir,
    snap_dir,
    input_shape,
    histogram_freq,
    write_graph,
    write_grads
)


# Train the model
model.fit_generator(
    generator=train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    # validation_data=val_generator,
    # validation_steps=validation_steps,
    callbacks=callbacks,
    use_multiprocessing=True,
    workers=threads
)

# Predicting configuration

In [ ]:
im_dir = r'../input/severstal-steel-defect-detection/test_images/'
model_path = r'./model.h5'
threshold = 0.5

# Start prediction

In [ ]:
# Load model
model = load_model(model_path, custom_objects={
    'dice_coef': dice_coef,
    'f1': f1,
    'jaccard_coef': jaccard_coef,
    'jaccard_coef_int': jaccard_coef_int,
    'categorical_focal_loss_fixed': categorical_focal_loss()
})

# Get all image files
all_items = os.listdir(im_dir)
image_files = [file for file in all_items if os.path.isfile(os.path.join(im_dir, file))]

imageid_classid = []
encodedpixels = []
for image_file in tqdm(image_files):
    im_name = image_file.split('.')[0]
    im_path = os.path.join(im_dir, image_file)

    # Get image data
#     image = cv2.imread(im_path, 1)
#     image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image = cv2.imread(im_path, 0)
    image = image.reshape(256, 1600, 1)
    image = image / 255.
    
    # Make prediction
    masks = model.predict(np.expand_dims(image, axis=0))[0]
    masks = (masks >= threshold).transpose(2, 0, 1).astype(np.uint8)
    
    
    # Save results
    for i, mask in enumerate(masks[1:]):
        if np.count_nonzero(mask) > 0:
            rle = mask2rle(mask)
        else:
            rle = np.nan

        imageid_classid.append('{}_{}'.format(image_file, i+1))
        encodedpixels.append(rle)


df = pd.DataFrame({
    'ImageId_ClassId': imageid_classid,
    'EncodedPixels': encodedpixels
})

df.to_csv('submission.csv', index=False)